In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr5051
Linux


In [3]:
path = hdfs_home+"/dataSets/spark-guide/flight-data/csv/2015-summary.csv"

In [4]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch02Introduction").getOrCreate()

In [7]:
print(spark.version)

2.3.0.2.6.5.0-292


In [8]:
sc = spark.sparkContext

In [9]:
spark

In [10]:
sc._conf.getAll()

[('spark.history.kerberos.keytab', 'none'),
 ('spark.eventLog.enabled', 'true'),
 ('spark.history.ui.port', '18081'),
 ('spark.driver.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.driver.appUIAddress', 'http://ip-172-31-38-146.ec2.internal:4042'),
 ('spark.driver.port', '33906'),
 ('spark.executor.extraLibraryPath',
  '/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 ('spark.executorEnv.PYTHONPATH',
  '{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.6-src.zip'),
 ('spark.history.provider',
  'org.apache.spark.deploy.history.FsHistoryProvider'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.filters',
  'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'),
 ('spark.app.name', 'ch02Introduction'),
 ('spark.eventLog.dir', 'hdfs:///spark2-history/'),
 ('spark.app.id', 'application_1552564637687_3

In [11]:
for x in sc._conf.getAll():
    if 'PROXY' in x[1]:
        print(x[1])

In [12]:
sc._jsc.sc().uiWebUrl().get()

'http://ip-172-31-38-146.ec2.internal:4042'

In [13]:
myRange = spark.range(1000).toDF("number")

In [14]:
myRange.take(3)

[Row(number=0), Row(number=1), Row(number=2)]

In [15]:
divisBy2 = myRange.where("number %2 = 0")

In [16]:
divisBy2.show()

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [17]:
pwd

'/home/kranthidr5051/projects/ml_lab/pyspark-lab/spark-tdg'

In [18]:
path

'/user/kranthidr5051/dataSets/spark-guide/flight-data/csv/2015-summary.csv'

In [19]:
flightData2015 = spark\
                .read\
                .option("inferSchema", True)\
                .option("header", True)\
                .csv(path)

In [20]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [21]:
flightData2015.sort("count").show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [22]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#23 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#23 ASC NULLS FIRST, 200)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#21,ORIGIN_COUNTRY_NAME#22,count#23] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://ip-172-31-35-141.ec2.internal:8020/user/kranthidr5051/dataSets/spark-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [23]:
spark.conf.set("spark.sql.shuffle.partitions", "5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [24]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(2) Sort [count#23 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#23 ASC NULLS FIRST, 5)
   +- *(1) FileScan csv [DEST_COUNTRY_NAME#21,ORIGIN_COUNTRY_NAME#22,count#23] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://ip-172-31-35-141.ec2.internal:8020/user/kranthidr5051/dataSets/spark-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>


In [25]:
spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES')

'http://ip-172-31-35-141.ec2.internal:8088/proxy/application_1552564637687_3059'

In [26]:
spark.conf.get('spark.driver.appUIAddress')

'http://ip-172-31-38-146.ec2.internal:4042'

In [27]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [28]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, COUNT(1) 
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME""")


dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()

In [29]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#21, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#21] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://ip-172-31-35-141.ec2.internal:8020/user/kranthidr5051/dataSets/spark-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#21, 5)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[partial_count(1)])
      +- *(1) FileScan csv [DEST_COUNTRY_NAME#21] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://ip-172-31-35-141.ec2.internal:8020/user/kranthidr5051/dataSets/spark-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAM

In [30]:
sqlWay.show()
dataFrameWay.show()

+--------------------+--------+
|   DEST_COUNTRY_NAME|count(1)|
+--------------------+--------+
|             Moldova|       1|
|             Bolivia|       1|
|             Algeria|       1|
|Turks and Caicos ...|       1|
|            Pakistan|       1|
|    Marshall Islands|       1|
|            Suriname|       1|
|              Panama|       1|
|         New Zealand|       1|
|             Liberia|       1|
|             Ireland|       1|
|              Zambia|       1|
|            Malaysia|       1|
|               Japan|       1|
|    French Polynesia|       1|
|           Singapore|       1|
|             Denmark|       1|
|               Spain|       1|
|             Bermuda|       1|
|            Kiribati|       1|
+--------------------+--------+
only showing top 20 rows

+--------------------+-----+
|   DEST_COUNTRY_NAME|count|
+--------------------+-----+
|             Moldova|    1|
|             Bolivia|    1|
|             Algeria|    1|
|Turks and Caicos ...|    1|
|  

In [31]:
spark.sql("SELECT max(count) FROM flight_data_2015").show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [32]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [33]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, SUM(count) AS destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY destination_total DESC
LIMIT 5""")

from pyspark.sql.functions import desc
maxDataFrame = flightData2015.groupBy("DEST_COUNTRY_NAME").sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)

In [34]:
maxSql.explain()
maxDataFrame.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#108L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#21,destination_total#108L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[sum(cast(count#23 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#21, 5)
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[partial_sum(cast(count#23 as bigint))])
         +- *(1) FileScan csv [DEST_COUNTRY_NAME#21,count#23] Batched: false, Format: CSV, Location: InMemoryFileIndex[hdfs://ip-172-31-35-141.ec2.internal:8020/user/kranthidr5051/dataSets/spark-gui..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>
== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#120L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#21,destination_total#120L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#21], functions=[sum(cast(count#23 as bigint))])
   +- Exchange hashpartitioning(DEST_C

In [35]:
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [36]:
maxDataFrame.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+

